## 1. Load Datasets



In [ ]:
!pip install bitsandbytes

import pandas as pd
import torch
from datasets import Dataset, DatasetDict
from transformers import MarianTokenizer, MarianMTModel, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

Train Dataset

In this section you can choose which dataset to use:
 - augment_dataset = True -> use the augmented dataset
 - augment_dataset = False -> use the original dataset

In [ ]:
augment_dataset = True

df1 = pd.read_csv("archaic_to_english_dataset.csv")
df1 = df1.rename(columns={"archaic": "source", "modern": "target"}) if "archaic" in df1.columns else df1
augmented
if augment_dataset:
  df2 = pd.read_csv("archaic_to_english_dataset_added.csv")
  df2 = df2.rename(columns={"archaic_italian": "source", "english_translation": "target"})
  df = pd.concat([df1, df2], ignore_index=True)
  df = df.drop_duplicates()
else:
  df = df1


train_dataset = DatasetDict({
    "train": Dataset.from_pandas(df.sample(frac=0.9, random_state=42).reset_index(drop=True)),
    "test": Dataset.from_pandas(df.sample(frac=0.1, random_state=42).reset_index(drop=True))
})

Test Dataset

In [ ]:
df = pd.read_csv("test.csv", sep=";")
df = df.rename(columns={"Sentence": "source", "Traductions": "target"})

test_dataset = Dataset.from_pandas(df)

## 2. Load Italian-to-English Model

In [ ]:
# Load the pre-trained MarianMT model and tokenizer for Italian-to-English translation
model_name = "Helsinki-NLP/opus-mt-it-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# HYPER-PARAMETERS
batch_size = 64
learning_rate = 1e-5
weight_decay = 0.001
epochs = 8

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:177: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(80379, 512, padding_idx=80378)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(80379, 512, padding_idx=80378)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

In [ ]:
# Preprocessing function to tokenize source and target texts
def preprocess(example):
    inputs = tokenizer(example["source"], truncation=True, padding="max_length", max_length=128)
    targets = tokenizer(example["target"], truncation=True, padding="max_length", max_length=128)
    inputs["labels"] = targets["input_ids"]
    return inputs

tokenized_dataset = train_dataset.map(preprocess, batched=True)
# Create a data collator suited for sequence-to-sequence models
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

Map:   0%|          | 0/106 [00:00<?, ? examples/s]

Map:   0%|          | 0/12 [00:00<?, ? examples/s]

## 3. Italian-to-English Model Training

fine-tuning using the choosen dataset

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./marian_finetuned",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    weight_decay=weight_decay,
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),
    logging_steps=10,
    report_to="none"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

<ipython-input-6-5bf51de8020b>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
# Let's Train ...
trainer.train()

Step,Training Loss
10,8.858100


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3465: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[80378]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


('./marian_finetuned_a/tokenizer_config.json',
 './marian_finetuned_a/special_tokens_map.json',
 './marian_finetuned_a/vocab.json',
 './marian_finetuned_a/source.spm',
 './marian_finetuned_a/target.spm',
 './marian_finetuned_a/added_tokens.json')

## 4. Load English-to-Italian Model

In [ ]:
model_en_it = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-it").to(device)
tokenizer_en_it = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-it")

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:177: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


## 5. Prometheus Evaluator
Since the PROMETHEUS model used with VLLM has 7 billion parameters, we attempted to load it using
the Hugging Face Transformers library and then quantize it in order to reduce memory usage and improve inference efficiency.   
To address this:

 - We used the Hugging Face Transformers library to load the model, as it provides a standardized interface for accessing pretrained weights and integrating them into existing pipelines.

 - We then applied quantization, a common technique that reduces the numerical precision of the model weights , with the goal of:

        - Lowering memory consumption

        - Speeding up inference

        - Maintaining reasonable accuracy



In [ ]:
from prometheus import PrometheusEval_AtM

evaluator = PrometheusEval_AtM(quantized = True, device = device)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

## 6. Translations

In [ ]:
import re
i = 1

source_sentences = []
predicted_sentences = []
gold_sentences = [] #annoted by hand
prometheus_score = []
df_col = ["source_sentences", "gold_sentences","predicted_sentences", "prometheus_score","GPT_score","user_score"]

print(len(test_dataset))
for sample in test_dataset:

    input_sentence = sample["source"]
    target_sentence = sample["target"]

    author = sample["Author"]
    date = sample["Date"]
    region = sample["Region"]

    # Archaic Italian -> Modern English
    inputs = tokenizer([input_sentence], return_tensors="pt", padding=True, truncation=True).to(device)
    translated_en_ids = model.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)
    translated_en = tokenizer.decode(translated_en_ids[0], skip_special_tokens=True)

    # Modern English -> Modern Italian
    inputs_en_it = tokenizer_en_it([translated_en], return_tensors="pt", padding=True, truncation=True).to(device)
    translated_it_ids = model_en_it.generate(**inputs_en_it, max_length=128, num_beams=4, early_stopping=True)
    translation = tokenizer_en_it.decode(translated_it_ids[0], skip_special_tokens=True)

    # Evaluate translation using Prometheus
    evaluation = evaluator.getEvaluation(input_sentence, translation, target_sentence)
    match_ = re.search(r'\[RESULT\]\s*(\d)', evaluation)
    if match_:
      result = int(match_.group(1))
    else:
      result = 0

    # Collect results
    source_sentences.append(input_sentence)
    predicted_sentences.append(translation)
    gold_sentences.append(target_sentence)
    prometheus_score.append(result)



    print(f"Sentence {i}")
    print(f"\tItaliano Arcaico            -> {input_sentence}")
    print(f"\tItaliano moderno            -> {translation}")
    print(f"\tGOLD LABEL                  -> {target_sentence}")
    print(f"\tPROMETHEUS EVALUATION       -> {result}")
    print(f"-----------------------------------------")
    i+=1


# Results to csv
z = [0 for _ in range(len(dataset))]
GPT_score,user_score = z,z
df = pd.DataFrame(list(zip(source_sentences, gold_sentences,predicted_sentences,prometheus_score,GPT_score,user_score)), columns=df_col)
df.to_csv("test_results.csv", sep=";", index=False)

10


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Sentence 1
	Italiano Arcaico            -> Et se l' occhio è nobile membro del corpo dell' uomo, dunque la salutazione è nobile parte della pistola, c' altressì allumina tutta la lettera come l' occhio allumina l' uomo.
	Italiano moderno            -> E se l'occhio è un nobile membro del corpo dell'uomo, allora il saluto è una parte nobile della pistola, c'è anche alluminio tutta la lettera come l'uomo occhio di alluminio.
	GOLD LABEL                  -> E se l’occhio è una parte nobile del corpo umano, allora il saluto è una parte nobile della lettera, che illumina l’intero testo come l’occhio illumina l’uomo.
	PROMETHEUS EVALUATION       -> 3
-----------------------------------------


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Sentence 2
	Italiano Arcaico            -> Tarentini, i quali erano nati di quegli di Lacedemonia et facta da lloro nobile cittade de' Greci.
	Italiano moderno            -> Tarentini, che nacquero da quelli di Lacedemonia et fata dalla loro nobile cittadella dei Greci.
	GOLD LABEL                  -> I Tarantini, nati dai lacedemoni, avevano fondato la loro nobile città greca.
	PROMETHEUS EVALUATION       -> 3
-----------------------------------------


...

Sentence 8
	Italiano Arcaico            -> pregollo che lo liberasse di quella obbligazione, in che egli l' aveva lasciato ubbligato. El gentile uomo assentì, e liberollo, e fecene carta.
	Italiano moderno            -> Lo supplico di liberarlo da quel legame, nel quale lo aveva lasciato legato; il gentiluomo si astentò, lo liberò e lo fece carta.
	GOLD LABEL                  -> Gli chiese di essere liberato da quell’obbligo che gli era stato imposto. Il nobile uomo acconsentì, lo liberò, e ne redasse un documento.
	PROMETHEUS EVALUATION       -> 3
-----------------------------------------


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Sentence 9
	Italiano Arcaico            -> L'oro verrà dall'Aquilone. Che figuriamo noi per l'Aquilone, se non il populo Gentile congelato dal freddo del peccato, il qual populo tenne sotto il giogo della sua tirannia
	Italiano moderno            -> L'oro uscirà dall'aquilone, che noi considereremo per l'aquilone, eccetto il popolo, Gentile, congelato dal freddo del peccato, che egli teneva sotto il giogo della sua tirannia.
	GOLD LABEL                  -> L’oro verrà dal Settentrione. Cosa rappresenta il Settentrione se non il popolo pagano, congelato dal freddo del peccato, che fu tenuto sotto il giogo della sua tirannia?
	PROMETHEUS EVALUATION       -> 1
-----------------------------------------


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Sentence 10
	Italiano Arcaico            -> sia in mezzo tra me e te: con noi non puo' tu già più lungamente dimorare, ch'io non lo sofferrò e non lo lascerò.
	Italiano moderno            -> stai in mezzo a me e a te; con noi non puoi più dimorare, perché io non l'ho rattristato e non lo lascerò.
	GOLD LABEL                  -> Sia in mezzo tra me e te: con noi non puoi più rimanere a lungo, perché io non lo permetterò e non lo tollererò oltre.
	PROMETHEUS EVALUATION       -> 2
-----------------------------------------
